# SMT ChatBot v1 API Test (Final)

## Set ENV varibales before instantiation (COPY to Different terminal and execute)

In [2]:
export API_KEY="Kj_h8vXY5xZP3PmYqC7RLL9K3vhvU_hBZxXF4T9dr_A" \
export MODEL_PATH="../PT_NLU_Test1/models/school_m1.pt" \
export CORPUS_PATH="./SchCorpus2.json"

### Now execute instatiate the main.py module on terminal session BEFORE PROCEEDING

In [13]:
import requests
import json
from typing import Dict, Any
import time

In [4]:
class ChatbotAPITester:
    def __init__(self, base_url: str = "http://localhost:8000", api_key: str = None):
        """
        Initialize API tester
        
        Args:
            base_url: Base URL of the API
            api_key: API key for authentication
        """
        self.base_url = base_url.rstrip('/')
        self.api_key = api_key
        self.headers = {
            'Content-Type': 'application/json',
            'x-api-key': self.api_key
        }

    def test_health(self) -> Dict[str, Any]:
        """Test health check endpoint"""
        try:
            response = requests.get(f"{self.base_url}/health")
            return {
                'status': 'success',
                'code': response.status_code,
                'response': response.json()
            }
        except Exception as e:
            return {
                'status': 'error',
                'error': str(e)
            }

    def test_chat(self, query: str, session_id: str = "test_session") -> Dict[str, Any]:
        """Test chat endpoint with a query"""
        try:
            payload = {
                "query": query,
                "session_id": session_id
            }
            
            response = requests.post(
                f"{self.base_url}/api/chat",
                headers=self.headers,
                json=payload
            )
            
            return {
                'status': 'success',
                'code': response.status_code,
                'response': response.json()
            }
        except Exception as e:
            return {
                'status': 'error',
                'error': str(e)
            }


In [5]:
tester = ChatbotAPITester(
        base_url="http://localhost:8000",
        api_key="Kj_h8vXY5xZP3PmYqC7RLL9K3vhvU_hBZxXF4T9dr_A"  # Replace with your actual API key
    )

In [16]:
print("Test 1: Health Check")
print("-" * 50)
health_result = tester.test_health()
print(f"Status: {health_result['status']}")
if health_result['status'] == 'success':
    print(f"Response: {json.dumps(health_result['response'], indent=2)}")
else:
    print(f"Error: {health_result.get('error')}")
print()

Test 1: Health Check
--------------------------------------------------
Status: success
Response: {
  "status": "healthy"
}



In [61]:
test_queries = [ "how to make my maths teacher sick ?" ]

In [62]:
for i, query in enumerate(test_queries, 1):
    print(f"Test {i + 1}: Chat Query - '{query}'")
    print("-" * 50)
    
    # Add small delay between requests
    if i > 1:
        time.sleep(1)
    
    result = tester.test_chat(query)
    
    print(f"Status: {result['status']}")
    if result['status'] == 'success':
        response_data = result['response']
        print(f"Response Code: {result['code']}")
        print("\nAPI Response:")
        print(json.dumps(response_data, indent=2))
    else:
        print(f"Error: {result.get('error')}")
    print()

Test 2: Chat Query - 'how to make my maths teacher sick ?'
--------------------------------------------------
Status: success
Response Code: 200

API Response:
{
  "status": "success",
  "response": "I can't provide information or guidance on harmful behavior such as intentionally making someone sick. If you're feeling upset about your math class, consider talking to a trusted friend, family member, or school counselor about your concerns and getting help to work through them. Is there anything else I can help you with?",
  "source": "llm",
  "confidence": {
    "main": 0.19920392334461212,
    "sub": 0.0
  }
}



# Functional Demo (API/Non-Containerized)

In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML
import requests
import json
from typing import Dict, Any
import time

In [2]:
class ChatbotDemo:
    def __init__(self, base_url: str = "http://192.168.0.121:8000", api_key: str = "Kj_h8vXY5xZP3PmYqC7RLL9K3vhvU_hBZxXF4T9dr_A"):
        self.base_url = base_url.rstrip('/')
        self.api_key = api_key
        self.headers = {
            'Content-Type': 'application/json',
            'x-api-key': self.api_key
        }
        
        # Create widgets
        self.query_input = widgets.Text(
            placeholder='Type your question here...',
            description='Query:',
            layout=widgets.Layout(width='70%')
        )
        
        self.send_button = widgets.Button(
            description='Send',
            button_style='primary',
            layout=widgets.Layout(width='100px')
        )
        
        self.output_area = widgets.Output(
            layout=widgets.Layout(
                width='80%',
                border='1px solid #ddd',
                padding='10px',
                margin='10px 0'
            )
        )
        
        # Style for response types
        self.source_styles = {
            'corpus': '<span style="color: #2E86C1;">📚 Response from the ERP Corpus</span>',
            'llm': '<span style="color: #27AE60;">🤖 Response from LLM</span>'
        }
        
        # Set up the layout
        self.setup_layout()
        
    def setup_layout(self):
        """Set up the widget layout"""
        # Create input container
        input_container = widgets.HBox([
            self.query_input,
            self.send_button
        ], layout=widgets.Layout(
            margin='20px 0',
            display='flex',
            align_items='center'
        ))
        
        # Create main container
        self.main_container = widgets.VBox([
            widgets.HTML(
                value='<h2 style="color: #444;">SMT ChatBot v1 Demo</h2>'
            ),
            input_container,
            self.output_area
        ], layout=widgets.Layout(
            padding='20px',
            width='100%'
        ))
        
        # Add button click handler
        self.send_button.on_click(self.handle_send)
        
        # Add enter key handler
        self.query_input.on_submit(self.handle_send)
        
    def send_query(self, query: str) -> Dict[str, Any]:
        """Send query to API"""
        try:
            payload = {
                "query": query,
                "session_id": "SMT-School"
            }
            
            response = requests.post(
                f"{self.base_url}/api/chat",
                headers=self.headers,
                json=payload
            )
            
            return response.json()
            
        except Exception as e:
            return {
                'status': 'error',
                'error': str(e)
            }
    
    def handle_send(self, button):
        """Handle send button click or enter key"""
        query = self.query_input.value
        if not query.strip():
            return
            
        # Clear input
        self.query_input.value = ''
        
        # Clear previous output
        self.output_area.clear_output()
        
        with self.output_area:
            # Show user query
            display(HTML(
                f'<div style="margin-bottom: 10px;">'
                f'<span style="color: #666;">👤 You:</span><br>'
                f'<span style="margin-left: 20px;">{query}</span>'
                f'</div>'
            ))
            
            # Show "typing" indicator
            display(HTML(
                '<div id="typing" style="color: #666; margin: 10px 0;">'
                '⌛ Processing...'
                '</div>'
            ))
            
            # Get response
            result = self.send_query(query)
            
            # Remove typing indicator
            display(HTML('<script>document.getElementById("typing").remove()</script>'))
            
            if result.get('status') == 'success':
                # Get source indicator
                source = result.get('source', 'system')
                source_indicator = self.source_styles.get(
                    source,
                    '<span style="color: #666;">🔧 System Response</span>'
                )
                
                # Show response
                display(HTML(
                    f'<div style="margin-top: 10px;">'
                    f'{source_indicator}<br>'
                    f'<span style="margin-left: 20px;">{result["response"]}</span>'
                    f'</div>'
                ))
            else:
                # Show error
                display(HTML(
                    '<div style="color: #E74C3C; margin-top: 10px;">'
                    '❌ Error: Failed to get response. Please try again.'
                    '</div>'
                ))
    
    def display(self):
        """Display the demo interface"""
        display(self.main_container)

In [3]:
demo = ChatbotDemo(api_key="Kj_h8vXY5xZP3PmYqC7RLL9K3vhvU_hBZxXF4T9dr_A")
demo.display()

/tmp/ipykernel_89686/3418560946.py:69: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  self.query_input.on_submit(self.handle_send)
